# **FDA Drug Shortage Data v3**

Run on Python 3.13 | No Errors | No Warnings

Drug Shortage Home Page: https://www.fda.gov/drugs/drug-safety-and-availability/drug-shortages

This program retrieves current drug shortage data directly from the FDA's Drug Shortages API, bypassing the need to manually download and clean a full CSV file. Retrieved data is stored in a local SQLite database. The program compares incoming records with existing entries and updates only those rows where the data has changed. This approach ensures the database remains current while avoiding redundant updates and unnecessary processing.

In [1]:
# Import packages

# For data manipulation
import pandas as pd

# For working with SQLite databases
import sqlite3

# for working with JSON data
import json

# For making HTTP requests and handling responses
import requests as req

In [2]:
# Load the SQL Magic extension
%load_ext sql

In [3]:
# Set SQL Magic options
%config SqlMagic.named_parameters = "enabled"
%config SqlMagic.feedback = False

In [4]:
# Create connection to the SQLite database
%sql sqlite:///DrugShortage.db

In [5]:
# Connect to the website and download the json file

# URL of the json file
url = 'https://api.fda.gov/drug/shortages.json?search=generic_name:"tirzepatide", "semaglutide", "dulaglutide", "liraglutide", "exenatide"&limit=100'

# Set headers to mimic a browser request
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

# Send a GET request
response = req.get(url, headers=headers)

# Check if the request was successful
if response.status_code == 200:
    # Parse the JSON content
    data = response.json()
    # Normalize the JSON data to a flat table
    if 'results' in data:
        df0 = pd.json_normalize(data['results'])
        print(df0.head())  # Display the first few rows
    else:
        print("No 'results' key found in the JSON response.")
else: # If the request was unsuccessful
    print(f"Failed to retrieve the json file. Status code: {response.status_code}")

  update_type initial_posting_date proprietary_name         strength  \
0  Reverified           2023-07-18      Liraglutide      [6 mg/1 mL]   
1     Revised           2022-03-31           Wegovy  [2.4 mg/.75 mL]   
2  Reverified           2023-07-18          Victoza      [6 mg/1 mL]   
3     Revised           2022-12-15        Trulicity   [4.5 mg/.5 mL]   
4     Revised           2022-03-31           Wegovy   [.25 mg/.5 mL]   

    package_ndc           generic_name  contact_info          availability  \
0  71288-563-84  Liraglutide Injection  844-824-8426             Available   
1  0169-4524-14  Semaglutide Injection  833-496-4689                   NaN   
2  0169-4060-12  Liraglutide Injection  833-493-4689  Limited Availability   
3  0002-3182-80  Dulaglutide Injection  800-545-5979                   NaN   
4  0169-4525-14  Semaglutide Injection  833-493-4689                   NaN   

  update_date        therapeutic_category  ...  \
0  2025-08-19  [Endocrinology/Metabolism]  ...  

In [6]:
# Display the first 5 rows of the dataframe
df0.head()

,update_type,initial_posting_date,proprietary_name,strength,package_ndc,generic_name,contact_info,availability,update_date,therapeutic_category,...,openfda.nui,openfda.pharm_class_epc,openfda.pharm_class_cs,openfda.pharm_class_moa,openfda.unii,change_date,resolved_note,related_info,shortage_reason,discontinued_date
0,Reverified,2023-07-18,Liraglutide,[6 mg/1 mL],71288-563-84,Liraglutide Injection,844-824-8426,Available,2025-08-19,[Endocrinology/Metabolism],...,"[N0000178480, M0160181, N0000020058]",[GLP-1 Receptor Agonist [EPC]],[Glucagon-Like Peptide 1 [CS]],[Glucagon-like Peptide-1 (GLP-1) Agonists [MoA]],[839I73S42A],NaN,NaN,NaN,NaN,NaN
1,Revised,2022-03-31,Wegovy,[2.4 mg/.75 mL],0169-4524-14,Semaglutide Injection,833-496-4689,NaN,2025-02-21,[Endocrinology/Metabolism],...,"[N0000178480, M0160181, N0000020058]",[GLP-1 Receptor Agonist [EPC]],[Glucagon-Like Peptide 1 [CS]],[Glucagon-like Peptide-1 (GLP-1) Agonists [MoA]],[53AXN4NNHX],2025-02-21,"Even when a medication is available, patients ...",NaN,NaN,NaN
2,Reverified,2023-07-18,Victoza,[6 mg/1 mL],0169-4060-12,Liraglutide Injection,833-493-4689,Limited Availability,2025-07-15,[Endocrinology/Metabolism],...,"[N0000178480, M0160181, N0000020058]",[GLP-1 Receptor Agonist [EPC]],[Glucagon-Like Peptide 1 [CS]],[Glucagon-like Peptide-1 (GLP-1) Agonists [MoA]],[839I73S42A],NaN,NaN,Estimated shortage duration TBD,Delay in shipping of the drug,NaN
3,Revised,2022-12-15,Trulicity,[4.5 mg/.5 mL],0002-3182-80,Dulaglutide Injection,800-545-5979,NaN,2025-06-13,[Endocrinology/Metabolism],...,"[N0000178480, M0160181, N0000020058]",[GLP-1 Receptor Agonist [EPC]],[Glucagon-Like Peptide 1 [CS]],[Glucagon-like Peptide-1 (GLP-1) Agonists [MoA]],[WTT295HSY5],2025-06-13,Available,NaN,NaN,NaN
4,Revised,2022-03-31,Wegovy,[.25 mg/.5 mL],0169-4525-14,Semaglutide Injection,833-493-4689,NaN,2025-02-21,[Endocrinology/Metabolism],...,"[N0000178480, M0160181, N0000020058]",[GLP-1 Receptor Agonist [EPC]],[Glucagon-Like Peptide 1 [CS]],[Glucagon-like Peptide-1 (GLP-1) Agonists [MoA]],[53AXN4NNHX],2025-02-21,"Even when a medication is available, patients ...",NaN,NaN,NaN


In [7]:
# Keep only the relevant columns
df0 = df0[['update_type', 'generic_name', 'update_date', 'presentation', 'company_name', 'status', 'availability', 'related_info', 'shortage_reason']]

In [8]:
# Rename the columns
df0.rename(columns={
    'update_type': 'Type of Update',
    'generic_name': 'Generic Name',
    'update_date': 'Date of Update',
    'presentation': 'Presentation',
    'company_name': 'Company Name',
    'status': 'Status',
    'availability': 'Availability Information',
    'related_info': 'Related Information',
    'shortage_reason': 'Reason for Shortage'
}, inplace=True)

In [9]:
# Reorder the columns
df0 = df0[['Generic Name', 'Company Name', 'Presentation', 'Type of Update', 'Date of Update', 'Availability Information', 'Related Information', 'Reason for Shortage', 'Status']]

In [10]:
# Verify the data types of the columns
df0.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24 entries, 0 to 23
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Generic Name              24 non-null     object
 1   Company Name              24 non-null     object
 2   Presentation              24 non-null     object
 3   Type of Update            24 non-null     object
 4   Date of Update            24 non-null     object
 5   Availability Information  9 non-null      object
 6   Related Information       6 non-null      object
 7   Reason for Shortage       2 non-null      object
 8   Status                    24 non-null     object
dtypes: object(9)
memory usage: 1.8+ KB


In [11]:
# Clean the data

# Replace the company names with shorter names
df0['Company Name'] = df0['Company Name'].replace({'Eli Lilly and Co.': 'Eli Lilly', 'Novo Nordisk, Inc.': 'Novo Nordisk', 'AstraZeneca AB': 'AstraZeneca'})

# Remove the word 'Injection' from the Generic Name column
df0['Generic Name'] = df0['Generic Name'].str.replace(' Injection', '')

# Split the Presentation column into three new columns: Brand Name, Administration, and Dosage
def split_presentation(pres):
    pres = pres.strip()
    parts = pres.split(',', maxsplit=2)
    if pres.startswith('Injection'):
        # Generic version
        brand_name = 'Liraglutide'
        administration = parts[0].strip()
        dosage = parts[1].strip() if len(parts) > 1 else ''
    else:
        # Brand-name version
        brand_name = parts[0].strip()
        administration = parts[1].strip() if len(parts) > 1 else ''
        dosage = parts[2].strip() if len(parts) > 2 else ''
    return pd.Series([brand_name, administration, dosage], index=['Brand Name', 'Administration', 'Dosage'])

# Apply the function to the dataframe
df0[['Brand Name', 'Administration', 'Dosage']] = df0['Presentation'].apply(split_presentation)

# Trim leading and trailing spaces from the new columns
df0['Brand Name'] = df0['Brand Name'].str.strip()
df0['Administration'] = df0['Administration'].str.strip()
df0['Dosage'] = df0['Dosage'].str.strip()

In [12]:
# Delete the Presentation column as it's no longer needed
df0.drop('Presentation', axis=1, inplace=True)

In [13]:
# Verify the changes
df0.head()

,Generic Name,Company Name,Type of Update,Date of Update,Availability Information,Related Information,Reason for Shortage,Status,Brand Name,Administration,Dosage
0,Liraglutide,"Meitheal Pharmaceuticals, Inc.",Reverified,2025-08-19,Available,NaN,NaN,Current,Liraglutide,Injection,6 mg/1 mL (NDC 71288-563-84)
1,Semaglutide,Novo Nordisk,Revised,2025-02-21,NaN,NaN,NaN,Resolved,Wegovy,Injection,2.4 mg/.75 mL (NDC 0169-4524-14)
2,Liraglutide,Novo Nordisk,Reverified,2025-07-15,Limited Availability,Estimated shortage duration TBD,Delay in shipping of the drug,Current,Victoza,Injection,6 mg/1 mL (NDC 0169-4060-12)
3,Dulaglutide,Eli Lilly,Revised,2025-06-13,NaN,NaN,NaN,Resolved,Trulicity,Injection,4.5 mg/.5 mL (NDC 0002-3182-80)
4,Semaglutide,Novo Nordisk,Revised,2025-02-21,NaN,NaN,NaN,Resolved,Wegovy,Injection,.25 mg/.5 mL (NDC 0169-4525-14)


In [14]:
# Further clean the data

# Drop ending period from Availibility Information column, if any values end with a period
df0['Availability Information'] = df0['Availability Information'].str.rstrip('.')

# Replace Currently avaliable with Available
df0['Availability Information'] = df0['Availability Information'].replace('Currently available', 'Available')

# If Status contains Discontinued, replace Availability Information with Discontinued
df0.loc[df0['Status'].str.contains('Discontinued'), 'Availability Information'] = 'Discontinued'

# If Administration contains Injectable, replace Administration with Injection
df0.loc[df0['Administration'].str.contains('Injectable'), 'Administration'] = 'Injection'

# If brand Name is Bydureon BCise or Byetta, replace Generic Name with Exenatide
df0.loc[df0['Brand Name'].str.contains('Bydureon Bcise|Byetta'), 'Generic Name'] = 'Exenatide'

# Drop ending period from Related Information column, if any values end with a period
df0['Related Information'] = df0['Related Information'].str.rstrip('.')

# If Status is Resolved, replace Availability Information with Resolved
df0.loc[df0['Status'] == 'Resolved', 'Availability Information'] = 'Resolved'

# If Availability Information is Available or Resolved and Related Information is blank, replace Related Information with custom text
df0.loc[(df0['Availability Information'] == 'Available') & (df0['Related Information'].isnull()), 'Related Information'] = 'Quantity limits may apply'

df0.loc[(df0['Availability Information'] == 'Resolved') & (df0['Related Information'].isnull()), 'Related Information'] = 'Availability may vary'

# Replace entries ending with TBD with Shortage duration TBD
df0['Availability Information'] = df0['Availability Information'].replace('Limited availability.  Estimated shortage duration TBD', 'Shortage duration TBD')

# Create a new column called Dose, which is the Dosage column with everything after mg removed
df0['Dose'] = df0['Dosage'].str.split(' mg').str[0] + ' mg'

# Remove the last 3 characters from the Dose column
df0['Dose'] = df0['Dose'].str[:-3]

# If Brand Name is Bydureon Bcise, replace Dose with 2
df0.loc[df0['Brand Name'] == 'Bydureon Bcise', 'Dose'] = '2'

# If Brand Name is Byetta, replace Dose with first three characters of Dosage
df0.loc[df0['Brand Name'] == 'Byetta', 'Dose'] = df0['Dosage'].str[:3]

# If Brand Name is Bydureon Bcise, Delete 'Extended Release, ' from the value in the Dosage column
df0.loc[df0['Brand Name'] == 'Bydureon Bcise', 'Dosage'] = df0['Dosage'].str.replace('Extended Release, ', '')

# If Brand Name is Byetta, delete '(250MCG/ML) ' from the value in the Dosage column
df0.loc[df0['Brand Name'] == 'Byetta', 'Dosage'] = df0['Dosage'].str.replace('(250MCG/ML) ', '')

# Convert the Dose column to numeric and replace any errors with NaN to make it easier to work with 
df0['Dose'] = pd.to_numeric(df0['Dose'], errors='coerce')

# If Company Name is AstraZeneca and Availibility Information is NaN, replace Related Information with 'Please select an alternative GLP-1 product'
df0.loc[(df0['Company Name'] == 'AstraZeneca') & (df0['Related Information'].isnull()), 'Related Information'] = 'No longer available'

In [15]:
# Convert 'Date of Update' to datetime format
df0["Date of Update"] = pd.to_datetime(df0["Date of Update"])
df0["Date of Update"] = df0["Date of Update"].dt.strftime('%Y-%m-%d')

In [16]:
# Sort on Brand Name, Date of Update, and Dose
df0.sort_values(by=['Brand Name', 'Date of Update', 'Dose'], inplace=True)

In [17]:
# Reindex the dataframe to reflect the new order
df0.reset_index(drop=True, inplace=True)

In [18]:
# Reorder the columns
df0 = df0[['Brand Name', 'Generic Name', 'Company Name', 'Administration', 'Dosage', 'Dose', 'Type of Update', 'Date of Update', 'Availability Information', 'Related Information','Reason for Shortage']]

In [19]:
# Verify the changes
df0.head()

,Brand Name,Generic Name,Company Name,Administration,Dosage,Dose,Type of Update,Date of Update,Availability Information,Related Information,Reason for Shortage
0,Bydureon Bcise,Exenatide,AstraZeneca,Injection,2 mg/.85 mL (NDC 0310-6540-04),2.0,New,2024-10-28,Discontinued,No longer available,NaN
1,Byetta,Exenatide,AstraZeneca,Injection,300MCG/1.2ML (NDC 0310-6512-01),300.0,New,2024-10-25,Discontinued,No longer available,NaN
2,Byetta,Exenatide,AstraZeneca,Injection,600MCG/2.4ML (NDC 0310-6524-01),600.0,New,2024-10-25,Discontinued,No longer available,NaN
3,Liraglutide,Liraglutide,Novo Nordisk,Injection,6 mg/1 mL (NDC 0480-3667-22),6.0,Reverified,2025-07-15,Available,Distributed by Teva,NaN
4,Liraglutide,Liraglutide,Novo Nordisk,Injection,6 mg/1 mL (NDC 0480-3667-20),6.0,Reverified,2025-07-15,Available,Distributed by Teva,NaN


In [20]:
# Determine if there are any updates or new additions (inserts) to the database

# Initialize counters and lists for updates and inserts
updates = 0
inserts = 0
updated_rows = []
inserted_rows = []

# Iterate through the cleaned dataset and prepare variables
for _, row in df0.iterrows():
    dosage = row["Dosage"]
    new_date = row["Date of Update"]

    brand = row["Brand Name"]
    generic = row["Generic Name"]
    company = row["Company Name"]
    admin = row["Administration"]
    dose = row["Dose"]
    update_type = row["Type of Update"]
    avail = row["Availability Information"]
    related = row["Related Information"]
    reason = row["Reason for Shortage"]

    # Check if Dosage already exists
    result = %sql SELECT [Date of Update] FROM SHORTAGE_DATA WHERE Dosage = :dosage

    if result and result[0][0]:  # Existing row found
        existing_date = pd.to_datetime(result[0][0])
        if pd.to_datetime(new_date) > existing_date:
            _ = %sql UPDATE SHORTAGE_DATA SET \
                [Brand Name]=:brand, [Generic Name]=:generic, [Company Name]=:company, Administration=:admin, \
                Dose=:dose, [Type of Update]=:update_type, [Date of Update]=:new_date, \
                [Availability Information]=:avail, [Related Information]=:related, [Reason for Shortage]=:reason \
                WHERE Dosage=:dosage
            updates += 1
            updated_rows.append((dosage, new_date))
    else:
        _ = %sql INSERT INTO SHORTAGE_DATA (\
            [Brand Name], [Generic Name], [Company Name], Administration, Dosage, \
            Dose, [Type of Update], [Date of Update], [Availability Information], \
            [Related Information], [Reason for Shortage]) VALUES (\
            :brand, :generic, :company, :admin, :dosage, :dose, :update_type, :new_date, \
            :avail, :related, :reason)
        inserts += 1
        inserted_rows.append((dosage, new_date))

In [21]:
# Display counts of updates and inserts
print(f"Updated rows: {updates}")
print(f"Inserted rows: {inserts}")

Updated rows: 2
Inserted rows: 0


In [22]:
# Display updated and inserted rows
if updated_rows:
    print("\n--- Updated Rows ---")
    for dosage, date in updated_rows:
        print(f"Updated: {dosage} → {date}")

if inserted_rows:
    print("\n--- Inserted Rows ---")
    for dosage, date in inserted_rows:
        print(f"Inserted: {dosage} → {date}")


--- Updated Rows ---
Updated: 6 mg/1 mL (NDC 71288-563-84) → 2025-08-19
Updated: 6 mg/1 mL (NDC 71288-563-85) → 2025-08-19


In [23]:
%%sql
-- Convert the 'Date of Update' column to the format YYYY-MM-DD
UPDATE SHORTAGE_DATA
SET [Date of Update] = 
    substr([Date of Update], 7, 4) || '-' || 
    substr([Date of Update], 1, 2) || '-' || 
    substr([Date of Update], 4, 2)
WHERE length([Date of Update]) = 10 AND instr([Date of Update], '/') > 0;

++
||
++
++

In [24]:
%%sql
-- Find the total count of duplicate rows in the SHORTAGE_DATA table
SELECT SUM(duplicate_count - 1) AS total_duplicates
FROM (
    SELECT COUNT(*) AS duplicate_count
    FROM SHORTAGE_DATA
    GROUP BY "Brand Name", "Generic Name", "Company Name", "Administration", "Dosage", "Dose", "Type of Update", "Date of Update", "Availability Information", "Related Information", "Reason for Shortage"
    HAVING COUNT(*) > 1
) as duplicates;

total_duplicates
None


In [25]:
%%sql
-- Delete duplicate rows in the SHORTAGE_DATA table
DELETE FROM SHORTAGE_DATA
WHERE rowid NOT IN (
    SELECT MIN(rowid)
    FROM SHORTAGE_DATA
    GROUP BY "Brand Name", "Generic Name", "Company Name", "Administration", "Dosage", "Dose", "Type of Update", "Date of Update", "Availability Information", "Related Information", "Reason for Shortage"
);

++
||
++
++

In [26]:
# Display the first 5 rows of the SHORTAGE_DATA table
%sql SELECT * FROM SHORTAGE_DATA LIMIT 5

Brand Name,Generic Name,Company Name,Administration,Dosage,Dose,Type of Update,Date of Update,Availability Information,Related Information,Reason for Shortage
Bydureon Bcise,Exenatide,AstraZeneca,Injection,2 mg/.85 mL (NDC 0310-6540-04),2.0,New,2024-10-28,Discontinued,No longer available,None
Byetta,Exenatide,AstraZeneca,Injection,300MCG/1.2ML (NDC 0310-6512-01),300.0,New,2024-10-25,Discontinued,No longer available,None
Byetta,Exenatide,AstraZeneca,Injection,600MCG/2.4ML (NDC 0310-6524-01),600.0,New,2024-10-25,Discontinued,No longer available,None
Liraglutide,Liraglutide,"Hikma Pharmaceuticals USA, Inc.",Injection,6 mg/1 mL (NDC 0143-9144-03),6.0,Reverified,2025-07-23,Available,Product will be made available as it is released,None
Liraglutide,Liraglutide,"Hikma Pharmaceuticals USA, Inc.",Injection,6 mg/1 mL (NDC 0143-9144-02),6.0,Reverified,2025-07-23,Available,Product will be made available as it is released,None


In [27]:
# Retrieve all data from the SHORTAGE_DATA table
df = %sql SELECT * FROM SHORTAGE_DATA

# Convert the ResultSet to a DataFrame
df1 = df.DataFrame()

In [28]:
# Display the first 5 rows of the dataframe (df1)
df1.head()

,Brand Name,Generic Name,Company Name,Administration,Dosage,Dose,Type of Update,Date of Update,Availability Information,Related Information,Reason for Shortage
0,Bydureon Bcise,Exenatide,AstraZeneca,Injection,2 mg/.85 mL (NDC 0310-6540-04),2.0,New,2024-10-28,Discontinued,No longer available,None
1,Byetta,Exenatide,AstraZeneca,Injection,300MCG/1.2ML (NDC 0310-6512-01),300.0,New,2024-10-25,Discontinued,No longer available,None
2,Byetta,Exenatide,AstraZeneca,Injection,600MCG/2.4ML (NDC 0310-6524-01),600.0,New,2024-10-25,Discontinued,No longer available,None
3,Liraglutide,Liraglutide,"Hikma Pharmaceuticals USA, Inc.",Injection,6 mg/1 mL (NDC 0143-9144-03),6.0,Reverified,2025-07-23,Available,Product will be made available as it is released,None
4,Liraglutide,Liraglutide,"Hikma Pharmaceuticals USA, Inc.",Injection,6 mg/1 mL (NDC 0143-9144-02),6.0,Reverified,2025-07-23,Available,Product will be made available as it is released,None


In [29]:
# Display basic information about the data
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36 entries, 0 to 35
Data columns (total 11 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Brand Name                36 non-null     object 
 1   Generic Name              36 non-null     object 
 2   Company Name              36 non-null     object 
 3   Administration            36 non-null     object 
 4   Dosage                    36 non-null     object 
 5   Dose                      36 non-null     float64
 6   Type of Update            36 non-null     object 
 7   Date of Update            36 non-null     object 
 8   Availability Information  36 non-null     object 
 9   Related Information       36 non-null     object 
 10  Reason for Shortage       2 non-null      object 
dtypes: float64(1), object(10)
memory usage: 3.2+ KB


In [30]:
# Write the cleaned data to a CSV file without the index if there are updates or inserts
if updates > 0 or inserts > 0:
    df1.to_csv('Drugshortages_cleaned.csv', index=False)
    print("CSV file updated.")
else:
    print("No changes detected. CSV not updated.")

CSV file updated.
